+ need to add possible features that appear to have an effect on demographic
# less social factors for now
* Focus on NE us and look at 10 yr span
* maybe work with NE data 2019 again but look at different pollutants
# Importing libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import klib

from IPython.core.display import display

# pd.set_option('display.max_columns', None)
# pd.reset_optio('max_rows')
#np.set_printoptions(threshold=sys.maxsize)

plt.style.use('dark_background')
plt.rcParams.update({"grid.linewidth":0.5, "grid.alpha":0.5})
sns.set(style='ticks', context='talk')

# Load data and constants

In [52]:
MA, CT, ME, NY, NH, NJ, PA, VT, DE, MD  = [25, 9, 23, 36, 33, 34, 42, 50, 10 , 24]
NORTH_EAST = [MA, CT, ME, NY, NH, NJ, PA, VT, DE, MD]
RI, FL,GA, NC, SC = [44, 12, 13, 37, 45] # midatlantic
AL, KY, MS, TN, WV = [1, 21, 29, 47, 45] # south east
IL, IN, IA, KS, MI,MN = [17, 18, 19, 20, 26, 27] # mid_west
daily_aqi_by_county_2017 = pd.read_csv('../../data/raw/daily_aqi_by_county_2017.csv')
daily_aqi_by_county_2018 = pd.read_csv('../../data/raw/daily_aqi_by_county_2018.csv')
daily_aqi_by_county_2019 = pd.read_csv('../../data/raw/daily_aqi_by_county_2019.csv')

county_demographic = pd.read_csv('../../data/raw/county_cc-est2019-alldata.csv', encoding='latin-1')

unemployment = pd.read_excel('../../data/raw/Unemployment.xls', header=0)
poverty_levels = pd.read_excel('../../data/raw/PovertyEstimates.xls', header=0)

annual_income_by_county = pd.read_csv('../../data/raw/CAINC1__ALL_AREAS_1969_2019.csv', encoding='latin-1')

def remove_col(df, col_name):
    '''Returns a dataframe with removed column or columns from old dataframe'''
    new_df = df.copy()

    if(type(col_name) != str and len(col_name) > 1):
        for index in col_name:
            new_df = new_df.drop(str(index), axis=1)
    else:
        new_df = new_df.drop(str(col_name), axis=1)
    return new_df

In [3]:
annual_aqi_by_county_2019 = pd.read_csv('../../data/raw/annual_aqi_by_county_2019.csv')

In [4]:
print(daily_aqi_by_county_2019.columns)

print(annual_aqi_by_county_2019.columns)
annual_aqi_by_county_2019.head()
# annual_aqi_by_county_2019.shape
annual_aqi_by_county_2019['Days with AQI'].min()

annual_aqi_by_county_2019.isna().sum()
annual_aqi_by_county_2019.head()

Index(['State Name', 'county Name', 'State Code', 'County Code', 'Date', 'AQI',
       'Category', 'Defining Parameter', 'Defining Site',
       'Number of Sites Reporting'],
      dtype='object')
Index(['State', 'County', 'Year', 'Days with AQI', 'Good Days',
       'Moderate Days', 'Unhealthy for Sensitive Groups Days',
       'Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days', 'Max AQI',
       '90th Percentile AQI', 'Median AQI', 'Days CO', 'Days NO2',
       'Days Ozone', 'Days SO2', 'Days PM2.5', 'Days PM10'],
      dtype='object')


,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10
0,Alabama,Baldwin,2019,271,237,34,0,0,0,0,80,52,37,0,0,220,0,51,0
1,Alabama,Clay,2019,107,97,10,0,0,0,0,67,50,30,0,0,0,0,107,0
2,Alabama,Colbert,2019,263,252,11,0,0,0,0,61,47,37,0,0,228,0,35,0
3,Alabama,DeKalb,2019,361,324,37,0,0,0,0,90,51,39,0,0,331,0,30,0
4,Alabama,Elmore,2019,228,208,20,0,0,0,0,100,50,39,0,0,228,0,0,0


In [5]:
hazard = annual_aqi_by_county_2019['Hazardous Days'] != 0

annual_aqi_by_county_2019.loc[hazard]

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10
21,Alaska,Fairbanks North Star,2019,365,230,96,18,16,1,4,377,103,38,0,13,130,1,215,6
35,Arizona,Pinal,2019,365,123,209,21,2,3,7,886,100,63,0,0,168,0,36,161
64,California,Inyo,2019,365,232,121,6,2,1,3,638,84,47,0,0,319,0,14,32
65,California,Kern,2019,365,119,151,87,6,1,1,548,122,67,0,2,260,0,73,30
66,California,Kings,2019,365,152,172,37,2,0,2,507,105,56,0,0,190,0,103,72
74,California,Mono,2019,365,270,80,6,0,0,9,3852,81,27,0,0,0,0,305,60
570,New Mexico,Dona Ana,2019,365,170,171,17,3,2,2,630,87,51,0,8,263,0,23,71
575,New Mexico,Luna,2019,361,349,10,1,0,0,1,617,29,12,0,0,0,0,0,361


* seems that pm5 and pm10 are common for days where aqi is above average

In [15]:
unemployment.columns
unemployment.head()
unemployment.shape
unemployment
# really need to clean this up looks like data from 2000 to 2019



,"Unemployment and median household income for the U.S., States, and counties, 2000-19",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87
0,Sources: Unemployment: U.S. Department of Labo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"For definitions of rural classifications, see ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"This table was prepared by USDA, Economic Rese...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,fips_txt,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
4,00000,US,United States,NaN,NaN,NaN,142601667,136904680,5696987,3.99504,...,161389026,155102319,6286707,3.89537,163100055,157115247,5984808,3.66941,65712,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3274,72145,PR,"Vega Baja Municipio, PR",1,1,1,19805,17607,2198,11.1,...,13117,11750,1367,10.4,13037,11791,1246,9.6,NaN,NaN
3275,72147,PR,"Vieques Municipio, PR",7,12,0,2441,2120,321,13.2,...,2718,2269,449,16.5,2585,2406,179,6.9,NaN,NaN
3276,72149,PR,"Villalba Municipio, PR",2,2,1,7636,6648,988,12.9,...,7311,6154,1157,15.8,7406,6231,1175,15.9,NaN,NaN
3277,72151,PR,"Yabucoa Municipio, PR",1,1,1,10613,9005,1608,15.2,...,8762,7509,1253,14.3,8691,7552,1139,13.1,NaN,NaN


In [49]:
poverty_levels # poverty levels in 2019

,"Poverty estimates for the U.S., States, and counties, 2019",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33
0,"Source: U.S. Department of Commerce, Bureau of...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"For definitions of rural classifications, see ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"This table was prepared by USDA, Economic Rese...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FIPStxt,Stabr,Area_name,Rural-urban_Continuum_Code_2003,Urban_Influence_Code_2003,Rural-urban_Continuum_Code_2013,Urban_Influence_Code_2013,POVALL_2019,CI90LBALL_2019,CI90UBALL_2019,PCTPOVALL_2019,CI90LBALLP_2019,CI90UBALLP_2019,POV017_2019,CI90LB017_2019,CI90UB017_2019,PCTPOV017_2019,CI90LB017P_2019,CI90UB017P_2019,POV517_2019,CI90LB517_2019,CI90UB517_2019,PCTPOV517_2019,CI90LB517P_2019,CI90UB517P_2019,MEDHHINC_2019,CI90LBINC_2019,CI90UBINC_2019,POV04_2019,CI90LB04_2019,CI90UB04_2019,PCTPOV04_2019,CI90LB04P_2019,CI90UB04P_2019
4,00000,US,United States,NaN,NaN,NaN,NaN,39490096,39248096,39732096,12.3,12.2,12.4,12000470,11865995,12134945,16.8,16.6,17,8258906,8160650,8357162,15.8,15.6,16,65712,65594,65830,3457689,3405854,3509524,18.2,17.9,18.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,56037,WY,Sweetwater County,5,8,5,8,3453,2743,4163,8.3,6.6,10,1075,771,1379,10.1,7.2,13,644,416,872,8.2,5.3,11.1,80639,73437,87841,NaN,NaN,NaN,NaN,NaN,NaN
3193,56039,WY,Teton County,7,8,7,8,1396,1073,1719,6,4.6,7.4,236,151,321,5.7,3.6,7.8,142,83,201,4.7,2.7,6.7,98837,86531,111143,NaN,NaN,NaN,NaN,NaN,NaN
3194,56041,WY,Uinta County,7,8,7,8,1699,1264,2134,8.5,6.3,10.7,541,346,736,9.6,6.1,13.1,336,193,479,7.8,4.5,11.1,70756,63191,78321,NaN,NaN,NaN,NaN,NaN,NaN
3195,56043,WY,Washakie County,7,11,7,11,845,626,1064,11.1,8.2,14,237,152,322,13.9,8.9,18.9,160,94,226,12.3,7.2,17.4,55122,50050,60194,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
pd.set_option('display.max_columns', 100)

poverty_levels.iloc[3:]

,"Poverty estimates for the U.S., States, and counties, 2019",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33
3,FIPStxt,Stabr,Area_name,Rural-urban_Continuum_Code_2003,Urban_Influence_Code_2003,Rural-urban_Continuum_Code_2013,Urban_Influence_Code_2013,POVALL_2019,CI90LBALL_2019,CI90UBALL_2019,PCTPOVALL_2019,CI90LBALLP_2019,CI90UBALLP_2019,POV017_2019,CI90LB017_2019,CI90UB017_2019,PCTPOV017_2019,CI90LB017P_2019,CI90UB017P_2019,POV517_2019,CI90LB517_2019,CI90UB517_2019,PCTPOV517_2019,CI90LB517P_2019,CI90UB517P_2019,MEDHHINC_2019,CI90LBINC_2019,CI90UBINC_2019,POV04_2019,CI90LB04_2019,CI90UB04_2019,PCTPOV04_2019,CI90LB04P_2019,CI90UB04P_2019
4,00000,US,United States,NaN,NaN,NaN,NaN,39490096,39248096,39732096,12.3,12.2,12.4,12000470,11865995,12134945,16.8,16.6,17,8258906,8160650,8357162,15.8,15.6,16,65712,65594,65830,3457689,3405854,3509524,18.2,17.9,18.5
5,01000,AL,Alabama,NaN,NaN,NaN,NaN,747478,730491,764465,15.6,15.2,16,233890,225712,242068,21.9,21.1,22.7,161670,154564,168776,20.7,19.8,21.6,51771,51179,52363,69236,65296,73176,24.2,22.8,25.6
6,01001,AL,Autauga County,2,2,2,2,6723,5517,7929,12.1,9.9,14.3,2040,1472,2608,15.9,11.5,20.3,1376,902,1850,14.4,9.4,19.4,58233,52517,63949,NaN,NaN,NaN,NaN,NaN,NaN
7,01003,AL,Baldwin County,4,5,3,2,22360,18541,26179,10.1,8.4,11.8,6323,4521,8125,13.5,9.6,17.4,4641,3295,5987,13.3,9.4,17.2,59871,54593,65149,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,56037,WY,Sweetwater County,5,8,5,8,3453,2743,4163,8.3,6.6,10,1075,771,1379,10.1,7.2,13,644,416,872,8.2,5.3,11.1,80639,73437,87841,NaN,NaN,NaN,NaN,NaN,NaN
3193,56039,WY,Teton County,7,8,7,8,1396,1073,1719,6,4.6,7.4,236,151,321,5.7,3.6,7.8,142,83,201,4.7,2.7,6.7,98837,86531,111143,NaN,NaN,NaN,NaN,NaN,NaN
3194,56041,WY,Uinta County,7,8,7,8,1699,1264,2134,8.5,6.3,10.7,541,346,736,9.6,6.1,13.1,336,193,479,7.8,4.5,11.1,70756,63191,78321,NaN,NaN,NaN,NaN,NaN,NaN
3195,56043,WY,Washakie County,7,11,7,11,845,626,1064,11.1,8.2,14,237,152,322,13.9,8.9,18.9,160,94,226,12.3,7.2,17.4,55122,50050,60194,NaN,NaN,NaN,NaN,NaN,NaN


,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2017,2018,2019
0,"""00000""",United States,,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,16937582000,17839255000,18542262000
1,"""00000""",United States,,CAINC1,2.0,...,Population (persons) 1/,Number of persons,324985539,326687501,328239523


In [58]:
saipe = pd.read_excel('../../data/raw/saipe_economic/est19all.xls')
saipe.head(6)

,Table with column headers in rows 3 and 4,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,2019 Poverty and Median Household Income Estim...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Geographic Information,NaN,NaN,NaN,All Ages,NaN,NaN,NaN,NaN,NaN,Age 0-17,NaN,NaN,NaN,NaN,NaN,Age 5-17 in Families,NaN,NaN,NaN,NaN,NaN,Median Household Income,NaN,NaN,Age 0-4,NaN,NaN,NaN,NaN,NaN
2,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Estimate, Age 0-17",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 0-17",90% CI Lower Bound,90% CI Upper Bound,"Poverty Estimate, Age 5-17 in Families",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 5-17 in Families",90% CI Lower Bound,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound,"Poverty Estimate, Age 0-4",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 0-4",90% CI Lower Bound,90% CI Upper Bound
3,00,000,US,United States,39490096,39248096,39732096,12.3,12.2,12.4,12000470,11865995,12134945,16.8,16.6,17,8258906,8160650,8357162,15.8,15.6,16,65712,65594,65830,3457689,3405854,3509524,18.2,17.9,18.5
4,01,000,AL,Alabama,747478,730491,764465,15.6,15.2,16,233890,225712,242068,21.9,21.1,22.7,161670,154564,168776,20.7,19.8,21.6,51771,51179,52363,69236,65296,73176,24.2,22.8,25.6
5,01,001,AL,Autauga County,6723,5517,7929,12.1,9.9,14.3,2040,1472,2608,15.9,11.5,20.3,1376,902,1850,14.4,9.4,19.4,58233,52517,63949,.,.,.,.,.,.


In [ ]:
# need to use us gov api to extract county educational attainment levels for 2019
